# Get stock data from API

In [1]:
import requests
import pandas as pd
import bokeh.plotting as bk
from bokeh.models import HoverTool, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool, Label
import numpy as np

API_URL = 'https://api.iextrading.com/1.0/stock/{}/chart/5y'

In [2]:
# Make the API request and load json data into DataFrame
symbol = 'amzn'
res = requests.get(API_URL.format(symbol))
data = res.json()
df = pd.DataFrame(data)

df.sample(10)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
926,-5.55,1.410107,-0.579,952.45,2017-08-24,959.00,"Aug 24, 17",941.140,957.42,5195726,5195726,949.8416
404,7.76,0.358233,1.467,536.76,2015-07-30,539.20,"Jul 30, 15",524.290,527.65,3743065,3743065,533.6289
6,-4.71,-0.004605,-1.183,393.37,2013-12-30,399.92,"Dec 30, 13",392.450,399.41,2487812,2487812,394.7291
333,13.95,-0.014373,3.714,389.51,2015-04-20,391.94,"Apr 20, 15",377.000,378.55,5016148,5016148,386.2131
280,9.94,-0.077735,2.804,364.47,2015-02-02,365.00,"Feb 2, 15",350.010,350.05,10231914,10231914,359.1882
745,19.02,0.921506,2.569,759.36,2016-12-05,761.49,"Dec 5, 16",742.000,745.00,4314723,4314723,756.0180
1048,19.66,2.715555,1.357,1468.35,2018-02-20,1488.77,"Feb 20, 18",1446.490,1446.49,6499228,6499228,1468.6700
342,1.09,0.070042,0.258,422.87,2015-05-01,425.64,"May 1, 15",416.000,423.82,3565824,3565824,420.5067
895,12.38,1.546901,1.245,1006.51,2017-07-12,1008.55,"Jul 12, 17",998.100,1000.65,3608574,3608574,1003.9800
127,-3.08,-0.179736,-0.941,324.16,2014-06-24,329.73,"Jun 24, 14",322.738,327.54,2627978,2627978,327.9327


In [3]:
#Add a new column in the shape of the original data
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [4]:
# Add a mid column that is midpoint between open and close
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [5]:
# Add height column after checking if close is greater than open
df['height'] = df.apply(lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.01, axis=1)

In [6]:
df.head(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs,mid,height
0,-0.77,0.000000,-0.194,395.19,2013-12-19,397.29,"Dec 19, 13",392.60,394.27,2429598,2429598,395.3560,0,394.730,0.92
1,7.01,0.017738,1.774,402.20,2013-12-20,404.72,"Dec 20, 13",395.78,396.55,5038537,5038537,401.1329,1,399.375,5.65


In [7]:
# some stuff to format plot
inc = df.close > df.open #Series where stocks closed higher than they opened
dec = df.close < df.open #Series in which stocks closed lower than they opened
w = .3     #line width for plot

In [8]:
#Define sources for increasing and decreasing stocks
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [9]:
#Define the Bokeh tools to include in the figure
hover = HoverTool(
    tooltips=[
        ('Date', '@date'),
        ('Low', '@low'),
        ('High', '@high'),
        ('Open', '@open'),
        ('Close', '@close'),
        ('Percent', '@changePercent')
    ]
)

In [10]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [11]:
#Define size and layout of figure
p = bk.figure(plot_width=1200, plot_height=800, title='Amazon', tools=TOOLS, toolbar_location='above')
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha = w
descriptor = Label(x=70,  y=70, text='Amazon Stock Price Over Time')
p.add_layout(descriptor)

In [12]:
#Create segments for price increases
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')
#Create segments for price decreases
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

GlyphRenderer(id='1048', ...)

In [13]:
#Create rects for price increases
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)
#Create rects for price decreases
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

GlyphRenderer(id='1060', ...)

In [17]:
# bk.show(p)
bk.save(p, './stock_plot.html')

'/home/scott/codefellows/401/projects/stock_portfolio/stock_plot.html'